In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import sys
import tqdm.notebook as tq
from collections import defaultdict
from collections import Counter
import nltk
from nltk.corpus import stopwords
import re
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

from urllib.parse import urlparse, unquote_plus

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
df_data = pd.read_csv('/kaggle/input/data16/final_output_last_v2.csv', encoding='utf-8')
df_data

,URL,content,Commercial/promotional,Content delivery,Entertainment,Error message,Gateway,Index,Informative,Journalistic,Scientific,Shopping
0,http://www.cybersexnetwork3.com/hardbodies/buk...,{'summary_text': 'We are currently manufacturi...,0,0,0,0,0,1,0,0,0,0
1,http://www.dirty-cartoonpictures.com/,{'summary_text': 'if you attended one of the f...,0,1,1,0,0,0,0,0,0,0
2,http://www.mrs-silk.com/topsites/topsites.html,"404 Not Found, Error: Name or service not known",0,0,0,0,0,1,0,0,0,0
3,http://www.findpics.net/,{'summary_text': '404 Not Found error was enco...,0,1,0,0,0,0,0,0,0,0
4,http://www.nakedpass.com/webmaster/webmasters.php,"404 Not Found, Error: Name or service not known",0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3264,http://www.oldwomen.org/,old nude women porn mature granny sex horny ol...,0,1,1,0,0,1,0,0,0,0
3265,http://www.webcamslave.com,bdsm cams bdsm chat bondage cams free bdsm vid...,0,1,1,0,0,1,0,0,0,0
3266,http://www.buyeuroporn.com/,porno dvd online european porn dvd cheap adult...,0,1,1,0,0,1,0,0,0,0
3267,http://www.analdreamhouse.com/30/03/agecheck/i...,anal dream house anal dream house anal dream h...,0,1,1,0,0,1,0,0,0,0


In [3]:
STOPWORDS = set(stopwords.words('english'))
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
REPLACE_IP_ADDRESS = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')


def extract_url(url):
    """
    Extract URL into meaningful words
    :param url: The web URL
    :return: The extracted string
    """
    parsed_unquoted = urlparse(unquote_plus(url))
    text = parsed_unquoted.netloc + ' ' + parsed_unquoted.path + ' ' + parsed_unquoted.params + ' ' + parsed_unquoted.query
    text = text.replace('\n', ' ').lower()
    text = REPLACE_IP_ADDRESS.sub(' ', text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub(' ', text)
    text = re.sub("www ", "", text)
    text = ' '.join([w for w in text.split() if w not in STOPWORDS])
    return text

In [4]:
df_data['URL'] = df_data['URL'].apply(extract_url)

In [5]:
# Combine title and abstract to increase power
df_data["combined"] = df_data["URL"] + ". " + df_data["content"]
df_data.drop(columns=["URL", "content"], axis=1, inplace=True)

In [6]:
type(df_data['combined'])

pandas.core.series.Series

In [7]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing punctuation and non-alphabetic tokens
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens if token.isalnum()]

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Join tokens back into a sentence
    processed_text = ' '.join(tokens)

    return processed_text


In [8]:
df_data['combined'] = df_data['combined'].apply(preprocess_text)

In [9]:
df_data

,Commercial/promotional,Content delivery,Entertainment,Error message,Gateway,Index,Informative,Journalistic,Scientific,Shopping,combined
0,0,0,0,0,0,1,0,0,0,0,cybersexnetwork com hardbodi bukkak index html...
1,0,1,1,0,0,0,0,0,0,0,dirti cartoonpictur com attend one follow inst...
2,0,0,0,0,0,1,0,0,0,0,mr silk com topsit topsit html found error na...
3,0,1,0,0,0,0,0,0,0,0,findpic net found error encount tri use errordocu
4,0,1,1,0,0,0,0,0,0,0,nakedpass com webmast webmast php found error...
...,...,...,...,...,...,...,...,...,...,...,...
3264,0,1,1,0,0,1,0,0,0,0,oldwomen org old nude women porn matur granni ...
3265,0,1,1,0,0,1,0,0,0,0,webcamslav com bdsm cam bdsm chat bondag cam f...
3266,0,1,1,0,0,1,0,0,0,0,buyeuroporn com porno dvd onlin european porn ...
3267,0,1,1,0,0,1,0,0,0,0,analdreamhous com agecheck index html anal d...


In [10]:
from sklearn.model_selection import train_test_split
# split into train and test
df_train, df_test = train_test_split(df_data, random_state=42, test_size=0.20, shuffle=True)
# split test into test and validation datasets
df_test, df_valid = train_test_split(df_test, random_state=42, test_size=0.50, shuffle=True)

In [11]:
# Hyperparameters
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 15
LEARNING_RATE = 1e-05
THRESHOLD = 0.5 # threshold for the sigmoid

In [12]:
from transformers import BertTokenizer, BertModel

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
df_train['combined']

1738    dept kent edu stuorg chang index htm  found er...
790           saman com uy  found error name servic known
2960    starquik com onlin groceri shop store mumbai b...
1022    rainbowsolut com transfer domain registrar hos...
1025      tritechsolut com  found error name servic known
                              ...                        
1095                     bsplayer org kill bill movi skin
1130          heeg de hmm squeak request url found server
1294        shadefound org  found error name servic known
860     mrc workshop hm content view    found error na...
3174    treemainten com tree care tree work tree surge...
Name: combined, Length: 2615, dtype: object

In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['combined'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'title': title
        }

In [16]:
target_list = list(df_data.columns)
target_list = target_list[:-1]
target_list

['Commercial/promotional',
 'Content delivery',
 'Entertainment',
 'Error message',
 'Gateway',
 'Index',
 'Informative',
 'Journalistic',
 'Scientific',
 'Shopping']

In [17]:
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN, target_list)
valid_dataset = CustomDataset(df_valid, tokenizer, MAX_LEN, target_list)
test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN, target_list)

In [18]:
# Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [19]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [20]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 10)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()

# # Freezing BERT layers: (tested, weaker convergence)
# for param in model.bert_model.parameters():
#     param.requires_grad = False

model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [21]:
# BCEWithLogitsLoss combines a Sigmoid layer and the BCELoss in one single class.
# This version is more numerically stable than using a plain Sigmoid followed
# by a BCELoss as, by combining the operations into one layer,
# we take advantage of the log-sum-exp trick for numerical stability.
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [22]:
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# Training of the model for one epoch
def train_model(training_loader, model, optimizer):

    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate droput, batch norm)
    model.train()
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader),
                      leave=True, colour='steelblue')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        # forward
        outputs = model(ids, mask, token_type_ids) # (batch,predict)=(32,13)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        # training accuracy, apply sigmoid, round (apply thresh 0.5)
        outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
        targets = targets.cpu().detach().numpy()
        correct_predictions += np.sum(outputs==targets)
        num_samples += targets.size   # total number of elements in the 2D array

        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

        # Update progress bar
        #loop.set_description(f"")
        #loop.set_postfix(batch_loss=loss)

    # returning: trained model, model accuracy, mean loss
    return model, float(correct_predictions)/num_samples, np.mean(losses)

In [24]:
def eval_model(validation_loader, model, optimizer):
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
            outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
            targets = targets.cpu().detach().numpy()
            correct_predictions += np.sum(outputs==targets)
            num_samples += targets.size   # total number of elements in the 2D array

    return float(correct_predictions)/num_samples, np.mean(losses)

In [25]:
data_dir = '/kaggle/working/'
history = defaultdict(list)
best_accuracy = 0

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    model, train_acc, train_loss = train_model(train_data_loader, model, optimizer)
    val_acc, val_loss = eval_model(val_data_loader, model, optimizer)

    print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f} train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    # save the best model
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), os.path.join(data_dir,"MLTC_model_state.bin"))
        best_accuracy = val_acc
    if epoch == EPOCHS:
        torch.save(model.state_dict(), os.path.join(data_dir,"MLTC_model_last_state.bin"))

Epoch 1/15


  0%|          | 0/327 [00:00<?, ?it/s]

train_loss=0.4249, val_loss=0.3489 train_acc=0.8195, val_acc=0.8612
Epoch 2/15


  0%|          | 0/327 [00:00<?, ?it/s]

train_loss=0.3290, val_loss=0.2979 train_acc=0.8719, val_acc=0.8914
Epoch 3/15


...

train_loss=0.0740, val_loss=0.3030 train_acc=0.9773, val_acc=0.8957
Epoch 14/15


  0%|          | 0/327 [00:00<?, ?it/s]

train_loss=0.0632, val_loss=0.3003 train_acc=0.9816, val_acc=0.9040
Epoch 15/15


  0%|          | 0/327 [00:00<?, ?it/s]

train_loss=0.0554, val_loss=0.3129 train_acc=0.9850, val_acc=0.9034


In [26]:
# Loading pretrained model (best model)
model = BERTClass()
model.load_state_dict(torch.load(os.path.join('/kaggle/working/',"MLTC_model_last_state.bin")))
model = model.to(device)

In [27]:
test_acc, test_loss = eval_model(test_data_loader, model, optimizer)

In [28]:
test_acc

0.9027522935779817

In [29]:
from sklearn.metrics import confusion_matrix, classification_report


In [30]:
def get_predictions(model, data_loader):
    """
    Outputs:
      predictions -
    """
    model = model.eval()

    titles = []
    predictions = []
    prediction_probs = []
    target_values = []

    with torch.no_grad():
      for data in data_loader:
        title = data["title"]
        ids = data["input_ids"].to(device, dtype = torch.long)
        mask = data["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data["targets"].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
        outputs = torch.sigmoid(outputs).detach().cpu()
        # thresholding at 0.5
        preds = outputs.round()
        targets = targets.detach().cpu()

        titles.extend(title)
        predictions.extend(preds)
        prediction_probs.extend(outputs)
        target_values.extend(targets)

    predictions = torch.stack(predictions)
    prediction_probs = torch.stack(prediction_probs)
    target_values = torch.stack(target_values)

    return titles, predictions, prediction_probs, target_values


In [31]:
titles, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)

In [32]:
predictions.numpy()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
print(classification_report(target_values, predictions, target_names=target_list))

                        precision    recall  f1-score   support

Commercial/promotional       0.69      0.77      0.73        96
      Content delivery       0.67      0.46      0.55        26
         Entertainment       0.91      0.77      0.83        26
         Error message       0.84      0.82      0.83        80
               Gateway       0.64      0.73      0.68       122
                 Index       0.93      0.85      0.88       117
           Informative       0.62      0.55      0.58        75
          Journalistic       0.88      0.57      0.70        40
            Scientific       0.95      0.72      0.82        29
              Shopping       0.87      0.59      0.70        22

             micro avg       0.76      0.72      0.74       633
             macro avg       0.80      0.68      0.73       633
          weighted avg       0.77      0.72      0.74       633
           samples avg       0.70      0.70      0.69       633



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
  return jaccard.mean()*100

In [35]:
print(j_score(target_values, predictions))

tensor(64.7248)
